In [1]:
using CSV, DataFrames, JuMP, Gurobi, StatsPlots, Random, Statistics

## Load in data

In [2]:
# Define sizes
#num_suppliers = 200 # i
#num_consumer_regions = 7 # j = c(i) 
#num_time = 10; # t 
#num_producer_regions = 5 # p(i)

# Parameters
# alpha
# delta

In [3]:
availability = CSV.read("availability.csv", DataFrame)
col_order = [1, 9, 8, 7, 6, 5, 4, 3, 2]
availability = availability[:, col_order]
availability = sort(availability, :Region)

Row,Region,yr_2016_shoes,yr_2017_shoes,yr_2018_shoes,yr_2019_shoes,yr_2020_shoes,yr_2021_shoes,yr_2022_shoes,yr_2023_shoes
,String15,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,AMERICAS,5.01118e6,5.27466e6,5.55227e6,5.84445e6,6.15248e6,6.47593e6,6.81695e6,7.17597e6
2,EMEA,7.5187e5,7.91713e5,8.33269e5,8.76967e5,9.23236e5,9.71647e5,1.02306e6,1.07704e6
3,N ASIA,2.10918e7,2.22018e7,2.33705e7,2.46005e7,2.58952e7,2.72584e7,2.86932e7,3.02033e7
4,S ASIA,7.66629e7,8.06977e7,8.4945e7,8.9416e7,9.41221e7,9.90759e7,1.0429e8,1.09779e8
5,SE ASIA,9.51028e7,1.00108e8,1.05377e8,1.10923e8,1.16761e8,1.22907e8,1.29376e8,1.36185e8


In [4]:
demand = CSV.read("demand.csv", DataFrame)
replace!(demand.Region, "Europe, Middle East, and Africa" => "EMEA")
demand

Row,Region,2016,2017,2018,2019,2020,2021,2022,2023
,String31,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Asia Pacific,1.84351e7,2.06687e7,2.24933e7,2.27891e7,2.17006e7,2.30219e7,2.58658e7,2.85839e7
2,EMEA,4.32876e7,4.45665e7,5.04292e7,5.40172e7,5.05751e7,5.98283e7,6.34163e7,7.09013e7
3,Greater China,2.2309e7,2.50644e7,3.00086e7,3.65837e7,3.97854e7,4.93391e7,4.64893e7,4.66524e7
4,Latin America,6.71511e6,7.52871e6,8.19335e6,8.30106e6,7.90458e6,8.38586e6,9.42178e6,1.04119e7
5,North America,7.98197e7,8.31245e7,8.00172e7,8.62232e7,8.00773e7,9.99485e7,1.04961e8,1.27871e8


In [5]:
revenue = CSV.read("rev.csv", DataFrame)
replace!(revenue.Region, "Europe, Middle East, and Africa" => "EMEA")
rev_year = []
for i in 2:size(revenue)[2]
    col_sum = sum(revenue[:, i]) * 1e6
    append!(rev_year, col_sum)
end
rev_year = Array(rev_year);

In [6]:
holding_costs = CSV.read("inventory_holding_cost.csv", DataFrame)[:, 2:end]

Row,Year,holding_cost_per_shoe
,Int64,Float64
1,2016,42.76
2,2017,49.47
3,2018,37.26
4,2019,43.52
5,2020,39.9
6,2021,33.37
7,2022,34.53
8,2023,43.01


In [7]:
shipping_costs = CSV.read("transport_costs.csv", DataFrame);

In [8]:
shipping_mapping = shipping_costs[:, 1:2]
shipping_mapping = sort(shipping_mapping, [:producer_region, :consumer_region], rev=[false, false])
shipping_mapping = hcat(DataFrame(Row_Count=1:nrow(shipping_mapping)), shipping_mapping)
shipping_mapping = combine(groupby(shipping_mapping, [:producer_region, :consumer_region])) do sub_df
    DataFrame(Value_mean = first(sub_df.Row_Count))
end
shipping_mapping = unstack(shipping_mapping, :consumer_region, :Value_mean)
shipping_mapping = sort(shipping_mapping, :producer_region)

Row,producer_region,Asia Pacific,Greater China,EMEA,North America,Latin America
,String15,Int64?,Int64?,Int64?,Int64?,Int64?
1,AMERICAS,1,3,2,5,4
2,EMEA,6,8,7,10,9
3,N ASIA,11,13,12,15,14
4,S ASIA,16,18,17,20,19
5,SE ASIA,21,23,22,25,24


In [9]:
production_costs = CSV.read("cost.csv", DataFrame)
replace!(production_costs.Region, "Europe, Middle East, and Africa" => "EMEA");
col_order = [1, 9, 8, 7, 6, 5, 4, 3, 2]
production_costs = production_costs[:, col_order]

Row,Region,yr_2016_shoes,yr_2017_shoes,yr_2018_shoes,yr_2019_shoes,yr_2020_shoes,yr_2021_shoes,yr_2022_shoes,yr_2023_shoes
,String15,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,AMERICAS,42.4736,46.8013,42.2206,45.3675,41.4347,45.7763,42.6593,53.6259
2,EMEA,42.8708,42.2413,39.3912,43.1394,42.7613,46.795,43.5146,51.5969
3,N ASIA,41.9221,44.7956,45.5397,44.6927,41.2893,48.4448,41.7661,48.7874
4,S ASIA,40.5973,39.8473,43.839,41.9903,39.0235,46.958,42.9565,53.8705
5,SE ASIA,41.1064,46.5148,45.8819,43.0424,39.6504,47.5189,44.3342,51.1256


In [10]:
production_costs

Row,Region,yr_2016_shoes,yr_2017_shoes,yr_2018_shoes,yr_2019_shoes,yr_2020_shoes,yr_2021_shoes,yr_2022_shoes,yr_2023_shoes
,String15,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,AMERICAS,42.4736,46.8013,42.2206,45.3675,41.4347,45.7763,42.6593,53.6259
2,EMEA,42.8708,42.2413,39.3912,43.1394,42.7613,46.795,43.5146,51.5969
3,N ASIA,41.9221,44.7956,45.5397,44.6927,41.2893,48.4448,41.7661,48.7874
4,S ASIA,40.5973,39.8473,43.839,41.9903,39.0235,46.958,42.9565,53.8705
5,SE ASIA,41.1064,46.5148,45.8819,43.0424,39.6504,47.5189,44.3342,51.1256


In [11]:
cost_of_shoes_init = 116.5
shoes_price_time = [116.5]
for i in 2:8
    shoes_price_minus_1 = shoes_price_time[i-1]
    shoe_price_current = shoes_price_minus_1 * 1.02
    append!(shoes_price_time, shoe_price_current)
end
shoes_price_time

8-element Vector{Float64}:
 116.5
 118.83
 121.2066
 123.630732
 126.10334664
 128.6254135728
 131.197921844256
 133.82188028114112

In [12]:
# Define sizes
num_suppliers = size(availability)[1] #i
num_consumer_regions = size(demand)[1] # j = c(i) 
num_time = size(availability)[2] - 1;  
num_producer_regions = size(availability)[1] # p(i)

5

### Set-up

In [13]:
# Sets
suppliers = 1:num_suppliers
consumer_regions = 1:num_consumer_regions
producer_regions = 1:num_producer_regions
years = 1:num_time
years_incl_zero = 0:num_time

0:8

In [14]:
# Parameters and input data
A = Matrix(availability[:, 2:end]) #5x8
D = Matrix(demand[:, 2:end]) #5x8
R = Array(rev_year) #8x1
H = Array(holding_costs[:, 2]) #8x1
T = Matrix(shipping_costs[:, 6:end]) #25x8
W = Matrix(shipping_mapping[:, 2:end]) #encodes the prodcuer/consumer region for T 5x5
C = Matrix(production_costs[:, 2:end]); #5x8

### Creating the model

In [15]:
# Define parameters (choose 1 value for now)
alpha = 1
cost_of_shoes = 116.5

116.5

In [16]:
function run_model(alpha, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C)
    # Initialize model
    model = Model(Gurobi.Optimizer);

    # Decision variables
    # X_{i,t} = quantity of shoes produced by supplier i (in producer region p(i)) at time t
    # S_{i,j,t} = quantity of shoes sold to consumer region j at time t, that are produced by supplier i (in producer region p(i))
    # E_{i,t} = holding quantity of shoes by supplier i (in producer region p(i)) at time t
    # M_{i,t} = marginal cost of getting supplier i to produce shoes at time t
    @variable(model, X[suppliers, years_incl_zero] >= 0);
    @variable(model, S[suppliers, consumer_regions, years] >= 0);
    @variable(model, E[suppliers, years_incl_zero] >= 0);
    @variable(model, M[consumer_regions, years] >= 0);


    # Objective function
    @objective(model, Max, 
    sum(sum(sum((shoes_price_time[t]-T[W[i,j],t]) * S[i,j,t] for i in suppliers) for j in consumer_regions) for t in years) 
    - sum(sum((C[i,t]) * X[i,t] + H[t] * E[i,t] + alpha * shoes_price_time[t] * M[i,t] for i in suppliers) for t in years))


    # Constraints
    @constraint(model, initial_production_constraint[i in suppliers], X[i,0] == 0);
    @constraint(model, initial_excess_constraint[i in suppliers], E[i,0] == 0);
    @constraint(model, production_sold_excess_relationship[i in suppliers, t in years], sum(S[i,j,t] for j in consumer_regions) == X[i,t] + E[i,t-1] - E[i,t]);
    @constraint(model, demand_constraint[j in consumer_regions, t in years], sum(S[i,j,t] for i in suppliers) <= D[j,t]);
    @constraint(model, supply_production_constraint[i in suppliers, t in years], X[i,t] <= A[i,t]);
    @constraint(model, unmet_demand[j in consumer_regions, t in years], M[j,t] == D[j,t] - sum(S[i,j,t] for i in suppliers)); # M[i,t] = max{X[i,t] − X[i,t−1], 0}

    #set_optimizer_attribute(model, "NonConvex", 2);
    optimize!(model);

    # Get values
    X_values = Matrix(value.(X))
    S_values = value.(S)
    E_values = Matrix(value.(E))
    M_values = Matrix(value.(M))

    return model, X_values, S_values, E_values, M_values
end

run_model (generic function with 1 method)

In [17]:
model_simple, X_values_simple, S_values_simple, E_values_simple, M_values_simple = run_model(0.5, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 170 rows, 330 columns and 810 nonzeros
Model fingerprint: 0xa37f940a
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+05, 1e+08]
Presolve removed 50 rows and 10 columns
Presolve time: 0.00s
Presolved: 120 rows, 320 columns, 755 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2744333e+35   3.840000e+32   1.274433e+05      0s
     126    1.3811851e+11   0.000000e+00   0.000000e+00      0s

Solved in 126 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.381185065e+11

User-callback calls 170, time in user-callback 0.00 sec


In [18]:
net_profit_simple = objective_value(model_simple)
holding_cost_simple = sum(H[t]*sum(E_values_simple[i,t] for i in suppliers) for t in years)
# sales_profit = net_profit + holding_cost

println("Net profit: ", net_profit_simple)
# println("Sales profit: ", sales_profit)
println("Holding cost: ", holding_cost_simple)

Net profit: 1.381185065437789e11
Holding cost: -6.408989429473877e-7


In [19]:
# Plot graphs

# Total costs over t
# Region

# Holding quantity over t

# Herfindalhs over t


In [20]:
# Plot graphs across alphas
# Ave Holding quantity over t
# Ave Herfindalhs over t


## Impose Integrality

In [21]:
function run_model_integer(alpha, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C)
    # Initialize model
    model = Model(Gurobi.Optimizer);

    # Decision variables
    # X_{i,t} = quantity of shoes produced by supplier i (in producer region p(i)) at time t
    # S_{i,j,t} = quantity of shoes sold to consumer region j at time t, that are produced by supplier i (in producer region p(i))
    # E_{i,t} = holding quantity of shoes by supplier i (in producer region p(i)) at time t
    # M_{i,t} = marginal cost of getting supplier i to produce shoes at time t
    @variable(model, X[suppliers, years_incl_zero] >= 0, Int);
    @variable(model, S[suppliers, consumer_regions, years] >= 0, Int);
    @variable(model, E[suppliers, years_incl_zero] >= 0,  Int);
    @variable(model, M[consumer_regions, years] >= 0,  Int);


    # Objective function
    @objective(model, Max, 
    sum(sum(sum((shoes_price_time[t]-T[W[i,j],t]) * S[i,j,t] for i in suppliers) for j in consumer_regions) for t in years) 
    - sum(sum((C[i,t]) * X[i,t] + H[t] * E[i,t] + alpha * shoes_price_time[t] * M[i,t] for i in suppliers) for t in years))


    # Constraints
    @constraint(model, initial_production_constraint[i in suppliers], X[i,0] == 0);
    @constraint(model, initial_excess_constraint[i in suppliers], E[i,0] == 0);
    @constraint(model, production_sold_excess_relationship[i in suppliers, t in years], sum(S[i,j,t] for j in consumer_regions) == X[i,t] + E[i,t-1] - E[i,t]);
    @constraint(model, demand_constraint[j in consumer_regions, t in years], sum(S[i,j,t] for i in suppliers) <= D[j,t]);
    @constraint(model, supply_production_constraint[i in suppliers, t in years], X[i,t] <= A[i,t]);
    @constraint(model, unmet_demand[j in consumer_regions, t in years], M[j,t] == D[j,t] - sum(S[i,j,t] for i in suppliers)); # M[i,t] = max{X[i,t] − X[i,t−1], 0}


    #set_optimizer_attribute(model, "NonConvex", 2);
    optimize!(model);

    

    # Get values
    X_values = Matrix(value.(X))
    S_values = value.(S)
    E_values = Matrix(value.(E))
    M_values = Matrix(value.(M))

    return model, X_values, S_values, E_values, M_values
end

run_model_integer (generic function with 1 method)

In [22]:
model_integer, X_values_integer, S_values_integer, E_values_integer, M_values_integer = run_model_integer(0.5, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 170 rows, 330 columns and 810 nonzeros
Model fingerprint: 0x8c4c0fef
Variable types: 0 continuous, 330 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+05, 1e+08]
Found heuristic solution: objective -1.08645e+11
Presolve removed 90 rows and 60 columns
Presolve time: 0.00s
Presolved: 80 rows, 270 columns, 505 nonzeros
Variable types: 0 continuous, 270 integer (0 binary)
Found heuristic solution: objective -8.59222e+10

Root relaxation: objective 1.381185e+11, 111 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Dep

In [23]:
net_profit_integer = objective_value(model_integer)
holding_cost_integer = sum(H[t]*sum(E_values_integer[i,t] for i in suppliers) for t in years)
# sales_profit = net_profit + holding_cost

println("Net profit: ", net_profit_integer)
# println("Sales profit: ", sales_profit)
println("Holding cost: ", holding_cost_integer)

Net profit: 1.3811850642419513e11
Holding cost: 129.03


#### Uncertainty in Availability
Define uncertainty based on random vector for reductions in availability (0-0.25)

In [24]:
random_seed = 123
Random.seed!(random_seed)
reduction_sets = []
for i in 1:8
    reductions = rand(10)/4
    append!(reduction_sets, [reductions])
end
reduction_sets

8-element Vector{Any}:
 [0.22657490969937025, 0.11087343311490114, 0.18641834528484852, 0.12802076000915358, 0.0634622722353774, 0.08353788409547971, 0.1068319702183998, 0.2168868000639895, 0.14547808559691142, 0.07786187517626322]
 [0.18664657921210961, 0.20480021089332517, 0.2385398156196156, 0.21147377058776504, 0.14668717513874724, 0.030453159481336173, 0.03444156044398475, 0.012702072814990473, 0.14489791663460896, 0.20096671475343672]
 [0.17078994140517822, 0.1895508153573528, 0.14295515043555873, 0.13475705210978084, 0.2412520282514698, 0.017677563402534957, 0.0826182988682938, 0.20156204088797453, 0.142728535882112, 0.014275914690461078]
 [0.06047046607640247, 0.11419112779429455, 0.13545158911383187, 0.07519194454809983, 0.1764925093192002, 0.2336974419360583, 0.00705954209984358, 0.061920756771170454, 0.044423767109071866, 0.051053017394202316]
 [0.19273732981823954, 0.1796666268975288, 0.12354912563271633, 0.014195510782407023, 0.05680259989325134, 0.11420297313570327, 0.196

In [25]:
function run_model_integer_uncertainty(alpha, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C)
    # Initialize model
    model = Model(Gurobi.Optimizer);

    # Decision variables
    # X_{i,t} = quantity of shoes produced by supplier i (in producer region p(i)) at time t
    # S_{i,j,t} = quantity of shoes sold to consumer region j at time t, that are produced by supplier i (in producer region p(i))
    # E_{i,t} = holding quantity of shoes by supplier i (in producer region p(i)) at time t
    # M_{i,t} = marginal cost of getting supplier i to produce shoes at time t
    @variable(model, X[suppliers, years_incl_zero] >= 0, Int);
    @variable(model, S[suppliers, consumer_regions, years] >= 0, Int);
    @variable(model, E[suppliers, years_incl_zero] >= 0,  Int);
    #@variable(model, M[consumer_regions, years] >= 0,  Int);
    @variable(model, M[suppliers, years] >= 0,  Int);
    # Define uncertain parameters



    # Objective function
    @objective(model, Max, 
    sum(sum(sum((shoes_price_time[t]-T[W[i,j],t]) * S[i,j,t] for i in suppliers) for j in consumer_regions) for t in years) 
    - sum(sum((C[i,t]) * X[i,t] + H[t] * E[i,t] + alpha * shoes_price_time[t] * M[i,t] for i in suppliers) for t in years))

    # Constraints
    @constraint(model, initial_production_constraint[i in suppliers], X[i,0] == 0);
    @constraint(model, initial_excess_constraint[i in suppliers], E[i,0] == 0);
    @constraint(model, production_sold_excess_relationship[i in suppliers, t in years], sum(S[i,j,t] for j in consumer_regions) == X[i,t] + E[i,t-1] - E[i,t]);
    @constraint(model, time_in_inventory[i in suppliers, t in years], sum(S[i,j,t] for j in consumer_regions) >= E[i,t-1]); #goods can't be in inventory more than 1 year
    @constraint(model, demand_constraint[j in consumer_regions, t in years], sum(S[i,j,t] for i in suppliers) <= D[j,t]);
    @constraint(model, supply_production_constraint[i in suppliers, t in years], X[i,t] <= A[i,t] - A[i,t] * maximum(reduction_sets[t])); #add uncertainty
    @constraint(model, unmet_demand[j in consumer_regions, t in years], M[j,t] == D[j,t] - sum(S[i,j,t] for i in suppliers));
    #@constraint(model, unused_availability[i in suppliers, t in years], M[i,t] >= A[i,t] - A[i,t] * maximum(reduction_sets[t]) - X[i, t]);

    #set_optimizer_attribute(model, "NonConvex", 2);
    optimize!(model);

    # Get values
    X_values = Matrix(value.(X))
    S_values = value.(S)
    E_values = Matrix(value.(E))
    M_values = Matrix(value.(M))

    return model, X_values, S_values, E_values, M_values
end

run_model_integer_uncertainty (generic function with 1 method)

In [26]:
model_uncertainty, X_values_uncertainty, S_values_uncertainty, E_values_uncertainty, M_values_uncertainty = run_model_integer_uncertainty(1, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 210 rows, 330 columns and 1050 nonzeros
Model fingerprint: 0x9448c301
Variable types: 0 continuous, 330 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+05, 1e+08]
Found heuristic solution: objective -2.17291e+11
Presolve removed 95 rows and 60 columns
Presolve time: 0.00s
Presolved: 115 rows, 270 columns, 715 nonzeros
Variable types: 0 continuous, 270 integer (0 binary)
Found heuristic solution: objective -1.87879e+11

Root relaxation: objective 8.741506e+10, 139 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  D

In [27]:
net_profit_uncertainty = objective_value(model_uncertainty)
holding_cost_uncertainty = sum(H[t]*sum(E_values_uncertainty[i,t] for i in suppliers) for t in 1:5)
# sales_profit = net_profit + holding_cost

println("Net profit: ", net_profit_uncertainty)
# println("Sales profit: ", sales_profit)
println("Holding cost: ", holding_cost_uncertainty)

Net profit: 8.741505696004437e10
Holding cost: 0.0


In [28]:
M_values_uncertainty

5×8 Matrix{Float64}:
 0.0        0.0        0.0        …  0.0        0.0        2.15437e7
 0.0        0.0        0.0           0.0        0.0        0.0
 1.69485e7  2.17511e7  2.41578e7     4.43064e7  3.20106e7  4.66524e7
 0.0        0.0        0.0           0.0        0.0        0.0
 0.0        0.0        0.0           0.0        0.0        0.0

In [29]:
A .- X_values_uncertainty[:, 2:end]

5×8 Matrix{Float64}:
 1.13541e6  1.25822e6  1.3395e6   …  1.52561e6  1.31332e6  1.72059e6
 1.70355e5  1.88856e5  2.01028e5     2.28903e5  1.97097e5  2.58244e5
 4.77887e6  5.29601e6  5.63818e6     6.42159e6  5.52786e6  7.2419e6
 1.73699e7  1.92496e7  2.04932e7     2.33405e7  2.00919e7  2.6322e7
 2.15479e7  2.38798e7  2.54224e7     2.89546e7  2.49248e7  3.26533e7

In [30]:
X_values_uncertainty

5×9 Matrix{Float64}:
 0.0       3.87577e6       4.01644e6  …       5.50364e6       5.45537e6
 0.0  581515.0        602857.0           825960.0        818794.0
 0.0       1.63129e7       1.69058e7          2.31653e7       2.29614e7
 0.0       5.9293e7        6.14481e7          8.41982e7       8.34574e7
 0.0       7.35549e7       7.62285e7          1.04451e8       1.03532e8

## Compare Models

In [31]:
#Outputs
models = ["Simple", "Integer Solution", "Integer + Uncertainty in Availability"]
net_profit = [net_profit_simple, net_profit_integer, net_profit_uncertainty]
holding_cost = [holding_cost_simple, holding_cost_integer, holding_cost_uncertainty]
output = DataFrame(Model = models, NetProfit = net_profit, HoldingCost = holding_cost)

Row,Model,NetProfit,HoldingCost
,String,Float64,Float64
1,Simple,1.38119e11,-6.40899e-7
2,Integer Solution,1.38119e11,129.03
3,Integer + Uncertainty in Availability,8.74151e10,0.0


In [32]:
#No uncertainty - only difference is the change in intial inventory
function run_model_after_shock(alpha, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C, X_values_pre, E_values_pre)
    # Initialize model
    model = Model(Gurobi.Optimizer);

    # Decision variables
    # X_{i,t} = quantity of shoes produced by supplier i (in producer region p(i)) at time t
    # S_{i,j,t} = quantity of shoes sold to consumer region j at time t, that are produced by supplier i (in producer region p(i))
    # E_{i,t} = holding quantity of shoes by supplier i (in producer region p(i)) at time t
    # M_{i,t} = marginal cost of getting supplier i to produce shoes at time t
    @variable(model, X[suppliers, years_incl_zero] >= 0, Int);
    @variable(model, S[suppliers, consumer_regions, years] >= 0, Int);
    @variable(model, E[suppliers, years_incl_zero] >= 0,  Int);
    @variable(model, M[consumer_regions, years] >= 0,  Int);


    # Objective function
    @objective(model, Max, 
    sum(sum(sum((shoes_price_time[t]-T[W[i,j],t]) * S[i,j,t] for i in suppliers) for j in consumer_regions) for t in years) 
    - sum(sum((C[i,t]) * X[i,t] + H[t] * E[i,t] + alpha * shoes_price_time[t] * M[i,t] for i in suppliers) for t in years))


    # Constraints
    @constraint(model, initial_production_constraint[i in suppliers], X[i,5] == X_values_pre[i,5]);
    @constraint(model, initial_excess_constraint[i in suppliers], E[i,5] == E_values_pre[i, 5]);
    @constraint(model, production_sold_excess_relationship[i in suppliers, t in years], sum(S[i,j,t] for j in consumer_regions) == X[i,t] + E[i,t-1] - E[i,t]);
    @constraint(model, demand_constraint[j in consumer_regions, t in years], sum(S[i,j,t] for i in suppliers) <= D[j,t]);
    @constraint(model, supply_production_constraint[i in suppliers, t in years], X[i,t] <= A[i,t]);
    @constraint(model, unmet_demand[j in consumer_regions, t in years], M[j,t] == D[j,t] - sum(S[i,j,t] for i in suppliers)); # M[i,t] = max{X[i,t] − X[i,t−1], 0}

    #set_optimizer_attribute(model, "NonConvex", 2);
    optimize!(model);

    

    # Get values
    X_values = Matrix(value.(X))
    S_values = value.(S)
    E_values = Matrix(value.(E))
    M_values = Matrix(value.(M))

    return model, X_values, S_values, E_values, M_values
end

run_model_after_shock (generic function with 1 method)

## Determine Best Parameters
Test various parameters on 2016-2020

In [33]:
#Punishment on unmet demand
alpha_values = [0, 0.1, 0.25, 0.5, 0.75, 0.9, 1];
length(alpha_values)

7

In [34]:
#severity of availability shock
delta_values = [0, 0.1, 0.2, 0.3, 0.4, 0.5];
#(1/0.3)

In [35]:
#Intialize Dictionaries for Results - 2016-2020
X_val = Dict()
S_val = Dict()
E_val = Dict()
M_val = Dict()

Dict{Any, Any}()

In [36]:
#Intialize Dictionaries for Results - 2021-2023
X_val_after = Dict()
S_val_after = Dict()
E_val_after = Dict()
M_val_after = Dict()

Dict{Any, Any}()

In [37]:
#Intialize Dataframe of values
results_16_20 = DataFrame(Model_Type = "", Alpha = 0.0, Net_Profit = 0.0, Holding_Cost = 0.0)
results_21_23 = DataFrame(Model_Type = "", Alpha = 0.0, Uncertainty_Range = 0.0, Net_Profit = 0.0, Holding_Cost = 0.0)

Row,Model_Type,Alpha,Uncertainty_Range,Net_Profit,Holding_Cost
,String,Float64,Float64,Float64,Float64
1,,0.0,0.0,0.0,0.0


In [38]:
for i in 1:length(alpha_values)
    alpha = alpha_values[i]
    #1. Run non-robust model
    model_integer, X_values_integer, S_values_integer, E_values_integer, M_values_integer = run_model_integer(alpha, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C);
    net_profit_integer = objective_value(model_integer)
    holding_cost_integer = sum(H[t]*sum(E_values_integer[z,t] for z in suppliers) for t in 1:5)
    #Add outputs to dictionary
    name = string(alpha) * "_non_robust"
    X_val[name] = X_values_integer
    S_val[name] = S_values_integer
    E_val[name] = E_values_integer
    M_val[name] = M_values_integer
    #Append to DataFrame
    push!(results_16_20, ["Non Robust", alpha, net_profit_integer, holding_cost_integer])
    #2. Run robust model
    model_uncertainty, X_values_uncertainty, S_values_uncertainty, E_values_uncertainty, M_values_uncertainty = run_model_integer_uncertainty(alpha, suppliers, consumer_regions, producer_regions, 1:5, 0:5, A, D, R, H, T, W, C);
    net_profit_uncertainty = objective_value(model_uncertainty)
    holding_cost_uncertainty = sum(H[t]*sum(E_values_uncertainty[z,t] for z in suppliers) for t in 1:5)
    #Add outputs to dictionary
    name = string(alpha) * "_robust"
    X_val[name] = X_values_uncertainty
    S_val[name] = S_values_uncertainty
    E_val[name] = E_values_uncertainty
    M_val[name] = M_values_uncertainty
    #Append to DataFrame
    push!(results_16_20, ["Robust", alpha, net_profit_uncertainty, holding_cost_uncertainty])
    #Evalute for various shocks in 2021-2023
    for j in 1:length(delta_values)
        A_shock = A
        for k in 1:5
            for l in 6:8
                Random.seed!(k+l)
                A_shock[k,l] = round(A_shock[k,l] * (1 - rand()/(1/delta_values[j])))
            end
        end
        #Test Non-Robust Model
        model_after_nonrobust, X_values_after_nonrobust, S_values_after_nonrobust, E_values_after_nonrobust, M_values_after_nonrobust = run_model_after_shock(alpha, suppliers, consumer_regions, producer_regions, 6:8, 5:8, A_shock, D, R, H, T, W, C, X_values_integer, E_values_integer);
        net_profit_after_nonrobust = objective_value(model_after_nonrobust)
        holding_cost_after_nonrobust = sum(H[t]*sum(E_values_after_nonrobust[z,t] for z in suppliers) for t in 1:3)
        push!(results_21_23, ["Non Robust", alpha, delta_values[j], net_profit_after_nonrobust, holding_cost_after_nonrobust])
        #Add outputs to dictionary
        name = string(alpha) * string(delta_values[j]) * "_non_robust"
        X_val_after[name] = X_values_after_nonrobust
        S_val_after[name] = S_values_after_nonrobust
        E_val_after[name] = E_values_after_nonrobust
        M_val_after[name] = M_values_after_nonrobust
        #Test Robust Model
        model_after_robust, X_values_after_robust, S_values_after_robust, E_values_after_robust, M_values_after_robust = run_model_after_shock(alpha, suppliers, consumer_regions, producer_regions,6:8, 5:8, A_shock, D, R, H, T, W, C, X_values_uncertainty, E_values_uncertainty);
        net_profit_after_robust = objective_value(model_after_robust)
        holding_cost_after_robust = sum(H[t]*sum(E_values_after_robust[z,t] for z in suppliers) for t in 1:3)
        push!(results_21_23, ["Robust", alpha, delta_values[j], net_profit_after_robust, holding_cost_after_robust])
        #Add outputs to dictionary
        name = string(alpha) * string(delta_values[j]) * "_robust"
        X_val_after[name] = X_values_after_robust
        S_val_after[name] = S_values_after_robust
        E_val_after[name] = E_values_after_robust
        M_val_after[name] = M_values_after_robust
    end

end
    

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 170 rows, 330 columns and 810 nonzeros
Model fingerprint: 0xff5357f1
Variable types: 0 continuous, 330 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+05, 1e+08]
Found heuristic solution: objective -0.0000000
Presolve removed 90 rows and 60 columns
Presolve time: 0.00s
Presolved: 80 rows, 270 columns, 505 nonzeros
Variable types: 0 continuous, 270 integer (0 binary)
Found heuristic solution: objective 1.278765e+10

Root relaxation: objective 1.381185e+11, 107 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth

In [39]:
results_16_20

Row,Model_Type,Alpha,Net_Profit,Holding_Cost
,String,Float64,Float64,Float64
1,,0.0,0.0,0.0
2,Non Robust,0.0,1.38119e11,0.0
3,Robust,0.0,6.68911e10,0.0
4,Non Robust,0.1,1.10961e11,9.47506e8
5,Robust,0.1,6.56828e10,0.0
6,Non Robust,0.25,8.51263e10,9.47506e8
7,Robust,0.25,6.38704e10,0.0
8,Non Robust,0.5,5.67707e10,3.36139e9
9,Robust,0.5,6.08498e10,0.0


In [40]:
results_21_23[25:51, :]

Row,Model_Type,Alpha,Uncertainty_Range,Net_Profit,Holding_Cost
,String,Float64,Float64,Float64,Float64
1,Robust,0.1,0.5,1.59631e10,0.0
2,Non Robust,0.25,0.0,9.91714e9,1.01542e9
3,Robust,0.25,0.0,6.11953e9,0.0
4,Non Robust,0.25,0.1,8.97713e9,1.01542e9
5,Robust,0.25,0.1,5.17952e9,0.0
6,Non Robust,0.25,0.2,7.16423e9,1.01542e9
7,Robust,0.25,0.2,3.36662e9,0.0
8,Non Robust,0.25,0.3,4.63788e9,1.01542e9
9,Robust,0.25,0.3,8.40277e8,0.0


### Try a Different Way
This time we try by assuming availability is known for 2016-2020 still, but assume there is assumed uncertainty around availability in 2021-2023.

In [41]:
function run_model_integer_withSomeUncertainty(alpha, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C, reduction_sets)
    # Initialize model
    model = Model(Gurobi.Optimizer);

    # Decision variables
    # X_{i,t} = quantity of shoes produced by supplier i (in producer region p(i)) at time t
    # S_{i,j,t} = quantity of shoes sold to consumer region j at time t, that are produced by supplier i (in producer region p(i))
    # E_{i,t} = holding quantity of shoes by supplier i (in producer region p(i)) at time t
    # M_{i,t} = marginal cost of getting supplier i to produce shoes at time t
    @variable(model, X[suppliers, years_incl_zero] >= 0, Int);
    @variable(model, S[suppliers, consumer_regions, years] >= 0, Int);
    @variable(model, E[suppliers, years_incl_zero] >= 0,  Int);
    #@variable(model, M[consumer_regions, years] >= 0,  Int);
    @variable(model, M[suppliers, years] >= 0,  Int);
    # Define uncertain parameters



    # Objective function
    @objective(model, Max, 
    sum(sum(sum((shoes_price_time[t]-T[W[i,j],t]) * S[i,j,t] for i in suppliers) for j in consumer_regions) for t in years) 
    - sum(sum((C[i,t]) * X[i,t] + H[t] * E[i,t] + alpha * shoes_price_time[t] * M[i,t] for i in suppliers) for t in years))

    # Constraints
    @constraint(model, initial_production_constraint[i in suppliers], X[i,0] == 0);
    @constraint(model, initial_excess_constraint[i in suppliers], E[i,0] == 0);
    @constraint(model, production_sold_excess_relationship[i in suppliers, t in years], sum(S[i,j,t] for j in consumer_regions) == X[i,t] + E[i,t-1] - E[i,t]);
    @constraint(model, time_in_inventory[i in suppliers, t in years], sum(S[i,j,t] for j in consumer_regions) >= E[i,t-1]); #goods can't be in inventory more than 1 year
    @constraint(model, demand_constraint[j in consumer_regions, t in years], sum(S[i,j,t] for i in suppliers) <= D[j,t]);
    @constraint(model, supply_production_constraint1[i in suppliers, t in 1:5], X[i,t] <= A[i,t]); #add uncertainty
    @constraint(model, supply_production_constraint2[i in suppliers, t in 6:8], X[i,t] <= A[i,t] - A[i,t] * maximum(reduction_sets[t])); #add uncertainty
    @constraint(model, unmet_demand[j in consumer_regions, t in years], M[j,t] == D[j,t] - sum(S[i,j,t] for i in suppliers));
    #@constraint(model, unused_availability[i in suppliers, t in years], M[i,t] >= A[i,t] - A[i,t] * maximum(reduction_sets[t]) - X[i, t]);

    #set_optimizer_attribute(model, "NonConvex", 2);
    optimize!(model);

    # Get values
    X_values = Matrix(value.(X))
    S_values = value.(S)
    E_values = Matrix(value.(E))
    M_values = Matrix(value.(M))

    return model, X_values, S_values, E_values, M_values
end

run_model_integer_withSomeUncertainty (generic function with 1 method)

In [42]:
rg = 1:5
maximum(rg)

5

In [43]:
function run_model_integer_afterShock(alpha, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C, X_values_input, S_values_input, E_values_input, M_values_input)
    # Initialize model
    model = Model(Gurobi.Optimizer);

    # Decision variables
    # X_{i,t} = quantity of shoes produced by supplier i (in producer region p(i)) at time t
    # S_{i,j,t} = quantity of shoes sold to consumer region j at time t, that are produced by supplier i (in producer region p(i))
    # E_{i,t} = holding quantity of shoes by supplier i (in producer region p(i)) at time t
    # M_{i,t} = marginal cost of getting supplier i to produce shoes at time t
    @variable(model, X[suppliers, years_incl_zero] >= 0, Int);
    @variable(model, S[suppliers, consumer_regions, years] >= 0, Int);
    @variable(model, E[suppliers, years_incl_zero] >= 0,  Int);
    #@variable(model, M[consumer_regions, years] >= 0,  Int);
    @variable(model, M[suppliers, years] >= 0,  Int);
    # Define uncertain parameters



    # Objective function
    @objective(model, Max, 
    sum(sum(sum((shoes_price_time[t]-T[W[i,j],t]) * S[i,j,t] for i in suppliers) for j in consumer_regions) for t in years) 
    - sum(sum((C[i,t]) * X[i,t] + H[t] * E[i,t] + alpha * shoes_price_time[t] * M[i,t] for i in suppliers) for t in years))

    # Constraints
    @constraint(model, initial_production_constraint[i in suppliers], X[i,0] == 0);
    @constraint(model, initial_excess_constraint[i in suppliers], E[i,0] == 0);
    #Given inputs - X all time periods
    @constraint(model, given_X[i in suppliers, t in years], X[i, t] == X_values_input[i, t]);
    #Given inputs - S, E, M for 1:5
    @constraint(model, given_S[i in suppliers, j in consumer_regions, t in 1:5], S[i, j, t] == S_values_input[i, j, t]);
    @constraint(model, given_E[i in suppliers, t in 1:5], E[i, t] == E_values_input[i, t]);
    @constraint(model, given_M[i in suppliers, t in 1:5], M[i, t] == M_values_input[i, t]);

    
    @constraint(model, production_sold_excess_relationship[i in suppliers, t in years], sum(S[i,j,t] for j in consumer_regions) == X[i,t] + E[i,t-1] - E[i,t]);
    @constraint(model, time_in_inventory[i in suppliers, t in years], sum(S[i,j,t] for j in consumer_regions) >= E[i,t-1]); #goods can't be in inventory more than 1 year
    @constraint(model, demand_constraint[j in consumer_regions, t in years], sum(S[i,j,t] for i in suppliers) <= D[j,t]);
    #@constraint(model, supply_production_constraint1[i in suppliers, t in 1:5], X[i,t] <= A[i,t]); #add uncertainty
    #@constraint(model, supply_production_constraint2[i in suppliers, t in 6:8], X[i,t] <= A[i,t] - A[i,t] * maximum(reduction_sets[t])); #add uncertainty
    @constraint(model, unmet_demand[j in consumer_regions, t in years], M[j,t] == D[j,t] - sum(S[i,j,t] for i in suppliers));
    #@constraint(model, unused_availability[i in suppliers, t in years], M[i,t] >= A[i,t] - A[i,t] * maximum(reduction_sets[t]) - X[i, t]);

    #set_optimizer_attribute(model, "NonConvex", 2);
    optimize!(model);

    # Get values
    X_values = Matrix(value.(X))
    S_values = value.(S)
    E_values = Matrix(value.(E))
    M_values = Matrix(value.(M))

    return model, X_values, S_values, E_values, M_values
end

run_model_integer_afterShock (generic function with 1 method)

In [44]:
#Intialize Dictionaries for Results
X_val_together = Dict()
S_val_together = Dict()
E_val_together = Dict()
M_val_together = Dict()

Dict{Any, Any}()

In [45]:
#Intialize Dataframe of values
results_together = DataFrame(Alpha = 0.0, Delta = 0.0, Net_Profit = 0.0, Holding_Cost = 0.0)

Row,Alpha,Delta,Net_Profit,Holding_Cost
,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0


In [46]:
#Test various parameters
for i in 1:length(alpha_values)
    alpha = alpha_values[i]
    for j in 1:length(delta_values)
        #Set up uncertainty set for this iteration
        random_seed = 123
        Random.seed!(random_seed)
        reduction_sets = []
        for k in 1:8
            if delta_values[j] == 0
                reductions = rand(10) * 0
            else
                reductions = rand(10)/(1/delta_values[j])
            end
            append!(reduction_sets, [reductions])
        end
        #Run model
        model, X_values, S_values, E_values, M_values = run_model_integer_withSomeUncertainty(alpha, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C, reduction_sets);
        net_profit = objective_value(model)
        holding_cost= sum(H[t]*sum(E_values[z,t] for z in suppliers) for t in years)
        #Add outputs to dictionary
        name = "Alpha_" * string(alpha) * "_Delta_" * string(delta_values[j])
        X_val_together[name] = X_values
        S_val_together[name] = S_values
        E_val_together[name] = E_values
        M_val_together[name] = M_values
        #Append to DataFrame
        push!(results_together, [alpha, delta_values[j], net_profit, holding_cost])
    end

end
    

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 210 rows, 330 columns and 1050 nonzeros
Model fingerprint: 0xbafedea9
Variable types: 0 continuous, 330 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+03, 1e+08]
Found heuristic solution: objective -0.0000000
Presolve removed 95 rows and 60 columns
Presolve time: 0.00s
Presolved: 115 rows, 270 columns, 715 nonzeros
Variable types: 0 continuous, 270 integer (0 binary)
Found heuristic solution: objective 1.278765e+10

Root relaxation: objective 8.011990e+10, 116 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Dep

In [47]:
results_together

Row,Alpha,Delta,Net_Profit,Holding_Cost
,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0
2,0.0,0.0,8.01199e10,1.46199e9
3,0.0,0.1,7.98174e10,1.46199e9
4,0.0,0.2,7.9515e10,1.46199e9
5,0.0,0.3,7.92125e10,1.46199e9
6,0.0,0.4,7.891e10,1.46199e9
7,0.0,0.5,7.86075e10,1.46199e9
8,0.1,0.0,7.12867e10,3.20193e9
9,0.1,0.1,7.09352e10,3.20193e9


In [48]:
#Intialize Dictionaries for Post-Shock
X_val_after = Dict()
S_val_after= Dict()
E_val_after = Dict()
M_val_after = Dict()

Dict{Any, Any}()

In [49]:
#Intialize Dataframe of values
results_after = DataFrame(Alpha = 0.0, Delta = 0.0, Shock_Delta = 0.0, Net_Profit = 0.0, Holding_Cost = 0.0)

Row,Alpha,Delta,Shock_Delta,Net_Profit,Holding_Cost
,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,0.0


In [50]:
#Test What Happens for Various Shocks
for i in 1:length(alpha_values)
    alpha = alpha_values[i]
    for j in 1:length(delta_values)
        delta = delta_values[j]
        #Get values:
        name = "Alpha_" * string(alpha) * "_Delta_" * string(delta)
        X_values_pre = X_val_together[name][:, 2:end]
        S_values_pre = S_val_together[name]
        E_values_pre = E_val_together[name][:, 2:end]
        M_values_pre = M_val_together[name]
        #shock delta
        for p in 1:length(delta_values)
            shock_delta = delta_values[p]
            #Set up uncertainty set for this iteration
            random_seed = 123
            Random.seed!(random_seed)
            A_shock = A
            X_values_update = X_values_pre
            for l in 6:8
                for k in 1:5
                    A_shock[k,l] = round(A_shock[k,l] * (1 - rand()/(1/shock_delta)))
                    if A[k, l] - X_values_update[k, l] < 0
                        X_values_update[k, l] = A[k, l]
                    end
                end
            end
            model, X_values, S_values, E_values, M_values = run_model_integer_afterShock(alpha, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C, X_values_update, S_values_pre, E_values_pre, M_values_pre);
            net_profit = objective_value(model)
            holding_cost= sum(H[t]*sum(E_values[z,t] for z in suppliers) for t in years)
            #Add outputs to dictionary
            name_after = "Alpha_" * string(alpha) * "_Delta_" * string(delta) * "_ShockDelta_" * string(shock_delta)
            X_val_after[name] = X_values
            S_val_after[name] = S_values
            E_val_after[name] = E_values
            M_val_after[name] = M_values
            #Append to DataFrame
            push!(results_after, [alpha, delta, shock_delta, net_profit, holding_cost])
        end
    end
end
    

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 385 rows, 330 columns and 1225 nonzeros
Model fingerprint: 0x67b73476
Variable types: 0 continuous, 330 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+03, 1e+08]
Found heuristic solution: objective 7.478958e+10
Presolve removed 385 rows and 330 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 8.01199e+10 7.47896e+10 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.011990354879e+10, best bound 8.011990354879e+10, gap 0.0000%

Use

In [51]:
results_after

Row,Alpha,Delta,Shock_Delta,Net_Profit,Holding_Cost
,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,8.01199e10,1.46199e9
3,0.0,0.0,0.1,7.99962e10,1.46199e9
4,0.0,0.0,0.2,7.97594e10,1.46199e9
5,0.0,0.0,0.3,7.94345e10,1.46199e9
6,0.0,0.0,0.4,7.90556e10,1.46199e9
7,0.0,0.0,0.5,7.86593e10,1.46199e9
8,0.0,0.1,0.0,7.86592e10,1.46199e9
9,0.0,0.1,0.1,7.85957e10,1.46199e9
